## Scripts for Creating the Mock Datasets

In [53]:
#importing necessary libraries
import pandas as pd
import random
from datetime import datetime, timedelta

#importing dataframes with pre-established .csv formats
cus_df = pd.read_csv('mock_data/mock_client.csv',sep=';')
acc_df = pd.read_csv('mock_data/mock_account.csv',sep=';')
trans_df = pd.read_csv('mock_data/mock_trans.csv',sep=';')

### Functions for Creating Customer Data
Customer data is the most independant dataset, and contains little overlap with the others. To create the data, each of the fields are randomised from a procured list of options. Using the script, as many customers as desired can be created. After the customer data is built, the account data can be created, followed by the transaction data.

### Customer Data

**client_id:**  The unique ID given to the customer.<br/>
**first_name:** The customer's first name. <br/>
**last_name:**  The customer's last name. <br/>
**dob:**        The customer's date of birth. <br/>
**join_date:**  The date the customer signed up to the bank. <br/>
**postcode:**   The customer's postcode (used for location identification). <br/>

In [54]:
def create_customer_data(cus_df,num_records):
    """
    This function generates a dataframe that contains a specified number of randomised customers.
    """
    
    i = 0
    
    while i < num_records:      
        #giving customers names
        cus_df.at[i,'first_name'] = choose_random(firstnames)
        cus_df.at[i,'last_name'] = choose_random(lastnames)
        
        #giving customers dates of birth
        cus_df.at[i,'dob'] = generate_datetime(1985,2001)
        
        #giving customers join date
        cus_df.at[i,'join_date'] = generate_datetime(cus_df.at[i,'dob'].year+18,2019)
        
        #giving customers postcodes
        cus_df.at[i,'postcode'] = choose_random(postcodes)
        
        #increment
        i+=1
        
    #sorting by date
    cus_df = cus_df.sort_values(by=['join_date'])
    
    #resetting index
    cus_df = cus_df.reset_index(drop=True)
    
    #loop to give ids in chronological order
    i = 0
    while i < cus_df.shape[0]:
        cus_df.at[i,'client_id'] = i+1
        i+=1
    
    #making account ID the index of the dataframe
    cus_df = cus_df.set_index("client_id", drop = True)
        
    return cus_df

### Account Data

**account_id:** Unique identifier for the account.<br/>
**client_id:** The customer to whom the accont belongs.<br/>
**account_name:** What the account is named (used for categorising purpose of account).<br/>
**date_created:** The date the account was created.<br/>
**balance:** The current balance of the account.<br/>

In [55]:
def create_account_data(acc_df,cus_df,num_records):
    """
    This function generates a dataframe that contains a specified number of randomised accounts based on
    a dataframe of customers.
    """
    
    i = 0
    
    while i < num_records:
        #assigning customer IDs to an account
        acc_df.at[i,'client_id'] = choose_random(cus_df.index.tolist())
        
        #giving account a name
        #TODO
        
        #giving account date of creation based on birth date of customer
        acc_df.at[i,'date_created'] = generate_datetime(cus_df.at[acc_df.at[i,'client_id'],'join_date'].year,2019)
        
        #giving account a balance
        acc_df.at[i,'balance'] = round(random.uniform(100,10000),2)
        
        i+=1
    
    
    #sorting by date
    acc_df = acc_df.sort_values(by=['date_created'])
    
    #resetting index
    acc_df = acc_df.reset_index(drop=True)
    
    #loop to give ids in chronological order
    i = 0
    while i < acc_df.shape[0]:
        acc_df.at[i,'account_id'] = i+1
        i+=1
    
    #making account ID the index of the dataframe
    acc_df = acc_df.set_index("account_id", drop = True)
    
    return acc_df

### Transaction Data
**trans_id:** The unique ID given to every new transaction.<br/>
**account_id:** The account ID of the person making the transaction.<br/>
**date:** The date the transaction occured.<br/>
**type:** The nature of the transaction.<br/>
**operation:** Whether the funds are ingoing or outgoing.<br/>
**amount:** The amount of money involved in the transaction.<br/>
**new_balance:** The balance after the transaction.<br/>
**account_to_or_from:** The account ID of the other party involved in the transaction (for transfers).<br/>

In [56]:
def create_transaction_data(trans_df,cus_df,acc_df,num_records):
    """
    This function generates a dataframe that contains a specified number of randomised transactions based on
    a dataframe of customers and accounts.
    """
    
    i = 0
    
    while i < num_records:
        #giving transaction an account number
        trans_df.at[i,'account_id'] = int(choose_random(acc_df.index.tolist()))
        
        #giving transaction a date
        trans_df.at[i,'date'] = generate_datetime(acc_df.at[int(trans_df.at[i,'account_id']),'date_created'].year,2019)
        
        #giving type of transaction
        trans_df.at[i,'type'] = choose_random(typelist)
        
        #giving operation of transaction
        if trans_df.at[i,'type'] == 'Income' or trans_df.at[i,'type'] == 'Transfer to Account':
            trans_df.at[i,'operation'] = 'ingoing'
        else:
            trans_df.at[i,'operation'] = 'outgoing'
        
        #giving transaction amount
        trans_df.at[i,'amount'] = round(random.uniform(1,100),2)
        
        #giving initial balance amount
        trans_df.at[i,'new_balance'] = acc_df.at[int(trans_df.at[i,'account_id']),'balance']
        
        i+=1
    
    #sorting by date
    trans_df = trans_df.sort_values(by=['date'])
    
    #resetting index
    trans_df = trans_df.reset_index(drop=True)
    
    #loop to give ids in chronological order
    i = 0
    while i < trans_df.shape[0]:
        trans_df.at[i,'trans_id'] = i+1
        #updating balance to reflect transaction
        if trans_df.at[i,'operation'] == 'ingoing':
            acc_df.at[int(trans_df.at[i,'account_id']),'balance'] = (acc_df.at[int(trans_df.at[i,'account_id']),'balance']  
                                                                     + trans_df.at[i,'amount'])
        else:
            acc_df.at[int(trans_df.at[i,'account_id']),'balance'] = (acc_df.at[int(trans_df.at[i,'account_id']),'balance']  
                                                                     - trans_df.at[i,'amount'])
            
        trans_df.at[i,'new_balance'] = acc_df.at[int(trans_df.at[i,'account_id']),'balance']
        
        i+=1
    
    #making transaction ID the index of the dataframe
    trans_df = trans_df.set_index("trans_id", drop = True)
    
    return trans_df

### Additional Functions and Declarations For Creating the Datasets

In [57]:
#declaring some lists to choose items from in the function calls above
#can be replaced with more comprehensive lists kept in files
firstnames = ['john','jack','jim','joe','james','julie','jules']
lastnames = ['black','white','brown','johnson','carmichael','vana','jones']
postcodes = [2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400]
typelist = ['Fuel','Groceries','Food','Entertainment','Shopping','Health','Uncategorised','Income',
            'Transfer to Account','Transfer from Account']

In [58]:
def choose_random(arr):
    """
    This function chooses a random index of an array
    """
    return arr[random.randint(0,len(arr)-1)]

def generate_datetime(min_year, max_year):
    """
    Generates a random time between two years
    Format: yyyy-mm-dd hh:mm:ss.000000
    """
    start = datetime(min_year, 1, 1, 00, 00, 00)
    years = max_year - min_year + 1
    end = start + timedelta(days=365 * years)
    return (start + (end - start) * random.random()).date()

### Testing the Data Generation

In [62]:
cus_df_temp = create_customer_data(cus_df,20)
cus_df_temp

first_name   last_name         dob   join_date postcode
client_id                                                        
1              julie       brown  1985-09-21  2007-09-09     2300
2              jules       brown  1991-06-06  2010-10-15     3300
3              james       brown  1991-12-04  2012-06-07     3300
4              jules       brown  1996-06-02  2014-02-02     2000
5                joe  carmichael  1994-07-04  2014-02-04     3200
6              julie  carmichael  1996-03-17  2014-02-15     3100
7                joe       black  1991-11-05  2014-04-16     3400
8              jules       black  1995-05-21  2015-01-28     3000
9              james       white  1995-12-17  2015-03-28     3000
10               joe       jones  1990-07-22  2015-07-05     3200
11               jim     johnson  1996-10-04  2015-12-02     2700
12             julie       black  1999-12-04  2017-07-14     3300
13              john     johnson  1996-08-25  2017-08-11     3300
14             james       black  1998-04-04  2017-11-24     3400
15             james        vana  1987-07-28  2018-04-15     2900
16               joe       white  1991-03-08  2018-05-11     2000
17             james       white  1995-08-31  2018-09-27     2800
18               jim       jones  2000-03-25  2018-12-19     3400
19              john        vana  1990-05-26  2019-08-25     2000
20               joe       white  2001-07-27  2019-11-28     2500

In [63]:
acc_df_temp = create_account_data(acc_df,cus_df_temp,20)
acc_df_temp

client_id account_name date_created  balance
account_id                                             
1                  1          NaN   2007-09-02   896.61
2                  1          NaN   2007-09-06  3659.63
3                  1          NaN   2009-07-03  2097.28
4                  4          NaN   2015-01-01  6660.23
5                  6          NaN   2016-06-25  4270.31
6                  7          NaN   2017-08-12  1255.76
7                 12          NaN   2017-10-15  3276.36
8                 12          NaN   2018-03-15  6717.04
9                 18          NaN   2018-11-13  9437.58
10                10          NaN   2018-12-12  7142.61
11                11          NaN   2019-01-24   152.28
12                18          NaN   2019-02-01  3581.98
13                18          NaN   2019-05-18  4273.12
14                 8          NaN   2019-06-10  1906.87
15                16          NaN   2019-07-04  5591.49
16                18          NaN   2019-07-06   6296.3
17                16          NaN   2019-09-12  6215.27
18                 9          NaN   2019-09-19   4642.3
19                16          NaN   2019-11-05  8161.58
20                15          NaN   2019-12-14  3230.18

In [64]:
trans_df_temp = create_transaction_data(trans_df,cus_df_temp,acc_df_temp,100)
trans_df_temp

account_id        date                   type operation amount  \
trans_id                                                                  
1                 2  2007-04-01              Groceries  outgoing  19.06   
2                 1  2008-02-22                   Food  outgoing  53.01   
3                 1  2008-10-29          Uncategorised  outgoing   7.93   
4                 2  2011-08-26    Transfer to Account   ingoing  50.15   
5                 1  2012-06-26                 Health  outgoing   23.5   
6                 1  2012-09-01                   Fuel  outgoing  59.92   
7                 3  2012-10-10              Groceries  outgoing  61.46   
8                 2  2012-11-07          Uncategorised  outgoing  19.92   
9                 3  2014-02-28              Groceries  outgoing  24.66   
10                3  2015-01-21                 Income   ingoing  45.95   
11                4  2015-05-30  Transfer from Account  outgoing  12.98   
12                5  2016-02-19                   Fuel  outgoing  30.38   
13                2  2016-04-09              Groceries  outgoing  83.13   
14                5  2016-10-03          Uncategorised  outgoing  57.81   
15                5  2016-11-17          Entertainment  outgoing  68.91   
16                6  2017-03-04          Entertainment  outgoing  59.26   
17                1  2017-03-17                   Food  outgoing  91.13   
18                6  2017-09-16                 Health  outgoing  20.41   
19                5  2017-10-08                 Health  outgoing  86.44   
20                7  2017-10-16              Groceries  outgoing  21.75   
21                3  2017-12-20                   Fuel  outgoing  41.13   
22                8  2018-01-02                   Food  outgoing  58.76   
23                9  2018-01-05              Groceries  outgoing  51.49   
24               10  2018-01-23          Entertainment  outgoing  80.57   
25                6  2018-04-24              Groceries  outgoing  48.45   
26               10  2018-06-09          Uncategorised  outgoing  33.89   
27                6  2018-06-10              Groceries  outgoing  43.41   
28               10  2018-08-01          Uncategorised  outgoing  55.74   
29               10  2018-08-24                 Income   ingoing  43.91   
30                6  2018-08-25              Groceries  outgoing  14.52   
...             ...         ...                    ...       ...    ...   
71               13  2019-08-05          Uncategorised  outgoing  30.52   
72               17  2019-08-10                 Income   ingoing  57.16   
73               17  2019-08-14                   Fuel  outgoing  29.49   
74               20  2019-08-16          Entertainment  outgoing  94.17   
75               16  2019-08-17                 Income   ingoing  36.01   
76               15  2019-08-20                   Food  outgoing  32.21   
77               10  2019-08-22    Transfer to Account   ingoing  77.74   
78               15  2019-09-03  Transfer from Account  outgoing   70.3   
79               13  2019-09-07  Transfer from Account  outgoing  10.88   
80                1  2019-09-08                   Food  outgoing  31.09   
81               20  2019-09-13                   Food  outgoing   29.1   
82               13  2019-09-18  Transfer from Account  outgoing  82.42   
83               10  2019-09-20              Groceries  outgoing  71.98   
84               19  2019-09-21                 Income   ingoing  96.96   
85               18  2019-09-24                 Health  outgoing  74.02   
86               14  2019-09-24               Shopping  outgoing  83.53   
87               20  2019-09-27                   Fuel  outgoing   9.44   
88                4  2019-10-01  Transfer from Account  outgoing  80.71   
89                3  2019-10-03  Transfer from Account  outgoing  60.39   
90               13  2019-10-09          Entertainment  outgoing   7.52   
91               20  2019-10-28   